https://llama-cpp-python.readthedocs.io/en/latest/

In [2]:
!pip install llama-cpp-python


[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/50.3 MB ? eta -:--:--
     ---------------------------------------- 0.1/50.3 MB 1.6 MB/s eta 0:00:32
     ---------------------------------------- 0.2/50.3 MB 2.9 MB/s eta 0:00:18
     ---------------------------------------- 0.4/50.3 MB 3.2 MB/s eta 0:00:16
      --------------------------------------- 0.7/50.3 MB 5.2 MB/s eta 0:00:10
     - -------------------------------------- 1.4/50.3 MB 6.1 MB/s eta 0:00:08
     - -------------------------------------- 2.0/50.3 MB 7.9 MB/s eta 0:00:07
     -- ------------------------------------- 2.7/50.3 MB 8.5 MB/s eta 0:00:06
     -- ------------------------------------- 3.4/50.3 MB 9.4 MB/s eta 0:00:05
     -- ------------------------------------- 3.8/50.3 MB 9.2 MB/s eta 0:00:06
     --- ------------------------------------ 4.5/50.3 MB 10.0 MB/s eta 0:00:05
     ---- ----------------------------------- 5.6/50.3 MB 11.2 MB/s eta 0:00:04
     ----- ---------------------------------- 6.6/50.3 M

In [1]:
from llama_cpp import Llama
max_tokens = 30000
# llama_llm = Llama(model_path="/Volumes/4Tera_SSD_2022/llm_models/llama-2-7b-chat.Q5_K_M.gguf", chat_format="llama-2")
# model_file_path = "/Users/jmcaffee/llm_models/Llama-3-8B-Instruct-32k-v0.1.Q4_K_M.gguf"
model_file_path = "C:\shared-drive\llm_models\Llama-3-8B-Instruct-32k-v0.1.Q4_K_M.gguf" #extremely fast on 3090
# model_file_path = "C:\shared-drive\llm_models\Meta-Llama-3-70B-Instruct.Q5_K_M.gguf" #wont load on rtx 3090
# model_file_path = "C:\shared-drive\llm_models\Meta-Llama-3-70B-Instruct.Q3_K_M.gguf" # this doesn't seem to work with python wrapper, but does if you run it via server.exe of llama.cpp
# model_file_path = "C:\shared-drive\llm_models\Meta-Llama-3-70B-Instruct.IQ2_XS.gguf" #doesn't respond at all on 3090
# allegedly iq2_xs.gguf will get 12 tokens/second on 3090: https://www.reddit.com/r/LocalLLaMA/comments/1caofxm/quick_speed_benchmark_for_llama_3_70b_on_1x_3090/
llm = Llama(model_path=model_file_path, chat_format="llama-3", n_ctx=max_tokens,  n_gpu_layers=81)
# llm.verbose = False # don't print a bunch of stuff

n_ubatch=32

# llm = Llama(model_path="/Volumes/4Tera_SSD_2022/llm_models/llama-2-13b-chat.Q5_K_M.gguf", chat_format="llama-2", n_ctx=2048) # n_ctx=2048
# function_llm = Llama(model_path="/Volumes/4Tera_SSD_2022/llm_models/functionary-7b-v1.Q5_K.gguf", chat_format="llama-2") # n_ctx=2048

ModuleNotFoundError: No module named 'llama_cpp'

In [2]:
import time
import json

def prompt(question):
    output = llm(
        f"Q: {question} A:", # Prompt
        # max_tokens=512,
        max_tokens=max_tokens,
        stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
        echo=False, # Echo the prompt back in the output
    )
    answer = output['choices'][0]['text']
    print(f"Q: {question} \nA:{answer}")
    return answer

def prompt2(question):
    start_time_seconds = time.time()
    output = llm.create_chat_completion(
        max_tokens=max_tokens,
        messages = [
            {
                "role": "system",
                "content": "You are an assistant who perfectly responds to questions with as many details as possible, without making up facts."
            },
            {
                "role": "user",
                "content": question
            }
        ]
    )
    elapsed_time_seconds = time.time() - start_time_seconds
    # print(output)
    answer = output['choices'][0]['message']['content']
    print(f'answer generated in {elapsed_time_seconds} seconds')
    print(f"Q: {question} \nA:{answer}")
    return answer

def stream_prompt(question):
    start_time_seconds = time.time()
    stream = llm.create_chat_completion(
        max_tokens=max_tokens,
        messages = [
            {
                "role": "system",
                "content": "You are an assistant who perfectly responds to questions with as many details as possible, without making up facts."
            },
            {
                "role": "user",
                "content": question
            }
        ],
        stream = True,
    )


    for output in stream:
        choices = output['choices']
        choice = choices[0]
        delta = choice['delta']
        if 'content' in delta:
            print(delta['content'], end='')
    print()

    elapsed_time_seconds = time.time() - start_time_seconds
    print(f'answer generated in {elapsed_time_seconds} seconds')

In [ ]:
stream_prompt("Where is paris?")

In [5]:
from projects.utils.with_time import with_time

@with_time
def get_n_words_from_text(text:str, number_of_words=1000):
    words = text.split()
    first_n_words = ' '.join(words[:number_of_words])
    return first_n_words

https://archive.org/stream/MajjhimaNikaya/MajjhimaNikayaSinglePaged._djvu.txt

In [13]:
with open('../../data/buddhist-texts/majjhima nikaya.txt', 'r') as file:
    large_text = file.read()
    
large_text = get_n_words_from_text(large_text, number_of_words=21000)    
stream_prompt(f"""
Using the text below, which is surrounded by <text> tags, summarize what is being discussed.  Include summaries of each sutta, the key points discussed, and then provide a summary of all the suttas, highlighting the key concepts and terms
<text>
{large_text}
</text>
""")    

get_n_words_from_text took 2.284121513366699e-05 ms to complete
The text appears to be a collection of Buddhist sutras or discourses from the Pali Canon, specifically from the Majjhima Nikaya (Middle Length Discourses) and the Digha Nikaya (Longer Discourses). The text consists of nine suttas (discourses) that discuss various topics related to the nature of reality, the path to enlightenment, and the characteristics of a noble disciple. Here is a summary of each sutta:
1. Mulapariyaya Sutta: This sutta discusses the root of all things, including the tendency to conceive oneself as earth, water, fire, air, beings, gods, and other phenomena, and how this conception leads to suffering.
2. Sabbasava Sutta: This sutta explains the importance of abandoning various taints (defilements) such as greed, hatred, delusion, and ignorance through seeing, restraining, using, enduring, avoiding, removing, and developing wholesome states.
3. Dhammadayada Sutta: This sutta emphasizes the importance of b

In [8]:
text = get_n_words_from_text(large_text, number_of_words=10000) 
print(text)

get_n_words_from_text took 9.279251098632812e-07 ms to complete
Part One The Root Fifty Discourses (MulapannasapaH) 1 The Division of the Discourse on the Root (Mulapariyayavagga) 1 Mulapariyaya Sutta The Root of All Things [1] 1. Thus have I heard. 1 On one occasion the Blessed One was living in Ukkattha in the Subhaga Grove at the root of a royal sala tree. There he addressed the bhikkhus thus: "Bhikkhus." 2 - "Venerable sir," they replied. The Blessed One said this: 2. "Bhikkhus, I shall teach you a discourse on the root of all things. 3 Listen and attend closely to what I shall say." - "Yes, venerable sir," the bhikkhus replied. The Blessed One said this: (the ordinary person) 3. "Here, bhikkhus, an untaught ordinary person, 4 who has no regard for noble ones and is unskilled and undisciplined in their Dhamma, who has no regard for true men and is unskilled and undisciplined in their Dhamma, perceives earth as earth. 5 Having perceived earth as earth, he conceives [himself as] eart